In [66]:
import importlib
import os
import sys
import warnings

import numpy as np

from music21 import environment

environment.set('midiPath', '/usr/bin/timidity')

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# Всё, что выводит CUDA / XLA, будет записано в файл
sys.stderr = open("xla_warnings.log", "w")



sys.path.append('src')

# в режиме редактирования сбрасуем кэшь иначе не видно изменений
for module in [
    "utils.music21_utils",
    "utils.data_utils",
    "utils.model_utils",
    "utils.generate_utils",
    "utils.train_utils",
    "utils.__init__"
    
]:
    imported_module = importlib.import_module(module)
    importlib.reload(imported_module)


from utils.__init__ import (
    midi_to_notes, split_parts, combine_to_score,
    normalize_sequence, denormalize_sequence, prepare_decoder_inputs,
    build_transformer,
    generate_autoregressive, generated_to_part,
    train_model,
    get_midis_by_compositor,
    get_notes_multitrack,
    split_melody_bass_chords_from_notes,
    prepare_sequences
)

In [10]:
сhopin_midis = get_midis_by_compositor("Chopin")

In [28]:
# все ноты
notes = get_notes_multitrack(сhopin_midis)
print(f"Извлечено {len(notes)} нот из {len(сhopin_midis)} файлов")

Извлечено 57894 нот из 48 файлов


In [30]:
# --- Разделяем инструменты ---
melody_part, bass_part, chord_part = split_melody_bass_chords_from_notes(notes)

In [32]:
# 2. Преобразование
melody_X = normalize_sequence([[n['pitch'], n['step'], n['duration']] for n in melody_part])[None, ...]
chord_y = normalize_sequence([[n['pitch'], n['step'], n['duration']] for n in chord_part])[None, ...]
dec_inputs, dec_targets = prepare_decoder_inputs(chord_y)

In [52]:
# Преобразуем партии в массивы признаков
melody_X, melody_y = prepare_sequences(melody_part)
bass_X, bass_y = prepare_sequences(bass_part)

In [54]:
# chord_part — это список словарей
chord_seq = [[n['pitch'], float(n['step']), float(n['duration'])] for n in chord_part]

In [56]:
chord_X, chord_y = prepare_sequences(chord_seq)

In [57]:
# --- Синхронизируем длины ---
min_len = min(len(melody_X), len(bass_X), len(chord_X))
melody_X, melody_y = melody_X[:min_len], melody_y[:min_len]
bass_X, bass_y = bass_X[:min_len], bass_y[:min_len]
chord_X, chord_y = chord_X[:min_len], chord_y[:min_len]

In [60]:
# вариант: конкат по времени (length = 2 * L_enc)
# --- Входы и выходы для трансформера ---
enc_inputs = np.concatenate([melody_X, bass_X], axis=1)
dec_inputs = np.zeros_like(chord_y)
dec_inputs[:, 1:, :] = chord_y[:, :-1, :]  # shift right
dec_targets = chord_y

In [61]:
# 3. Модель
model = build_transformer()
train_model(model, dec_inputs, dec_inputs, dec_targets, epochs=10)

Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 68s 299ms/step - loss: 0.3330 - val_loss: 0.0050
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0238 - val_loss: 0.0048
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0107 - val_loss: 0.0050
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0072 - val_loss: 0.0044
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0058 - val_loss: 0.0025
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0044 - val_loss: 0.0022
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0040 - val_loss: 0.0023
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0038 - val_loss: 0.0023
Epoch 9/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0037 - val_loss: 0.0022
Epoch 10/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0036 - val_loss: 0.0022


In [65]:
# 4. Генерация
gen = generate_autoregressive(model, melody_X, dec_inputs[:, :4, :], length=32)
gen_denorm = denormalize_sequence(gen)
part = generated_to_part(gen_denorm, make_chords=True)

In [67]:
combine_to_score([part]).show('midi')

Playing /tmp/tmpdq4ft2hj.mid
MIDI file: /tmp/tmpdq4ft2hj.mid
Format: 1  Tracks: 2  Divisions: 10080
Track name: Piano
Playing time: ~12 seconds
Notes cut: 0
Notes lost totally: 0


In [64]:
combine_to_score([part]).write('midi', fp='chopin_rest_by_first_transformer.midi')

'chopin_rest_by_first_transformer.midi'